# Dataset Creation
We need a dataset that has the  6 attributes we are using for classifying a player's skill - 
<ul>
    <li>Runs Scored</li>
    <li>Batting Average</li>
    <li>Batting Strike Rate</li>
    <li>Wickets Taken</li>
    <li>Bowling Average</li>
    <li>Bowling Strike Rate</li>
</ul>

From [ESPN Cricinfo's](https://stats.espncricinfo.com/ci/engine/records/index.html) stats website we can find some of this for each player. This can be scraped using a few libraries that can parse HTML elements.
<b><br>We shall look at the top 50 batsmen and top 50 bowlers in One Day Internationals of the past decade, from 2010 to 2019.</b>

While most of the dataset creation process is scripted, some part has to be manual:
<ul>
    <li>For batsmen: Wickets Taken, Bowling Average and Bowling Striker Rate attributes.</li>
    <li>For bowlers: Runs Scored, Batting Average and Batting Strike Rate attributes.</li>
</ul>
This is because I couldn't find all the 6 attributes on a site for cricketers in any stats website. 

Let us first start with scraping the stats for the top 50 ODI batsmen (in terms of runs scored) in 2010-2019.

In [7]:
import requests
import lxml.html as lh
import pandas as pd

In [83]:
page = requests.get('https://stats.espncricinfo.com/ci/engine/records/batting/most_runs_career.html?class=2;id=201;type=decade')

In [84]:
#store contents of the website under doc
doc = lh.fromstring(page.content)

In [85]:
#to parse data stored between <tr> tags
tr_elements = doc.xpath('//tr')

In [87]:
#create empty list
col = []
i = 0

for t in tr_elements[0]:
    i+=1
    name = t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Player"
2:"Span"
3:"Mat"
4:"Inns"
5:"NO"
6:"Runs"
7:"HS"
8:"Ave"
9:"BF"
10:"SR"
11:"100"
12:"50"
13:"0"
14:"4s"
15:"6s"


In [88]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=15:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [89]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [90]:
df

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
0,V Kohli (INDIA),2010-2019,227,220,37,11125,183,60.79,11821,94.11,42,52,13,1038,115
1,RG Sharma (INDIA),2010-2019,180,176,22,8249,264,53.56,9101,90.63,28,39,10,750,233
2,HM Amla (SA),2010-2019,159,157,11,7265,159,49.76,8152,89.11,26,33,4,738,48
3,AB de Villiers (SA),2010-2018,135,129,28,6485,176,64.20,5908,109.76,21,33,2,520,155
4,LRPL Taylor (NZ),2010-2019,155,145,26,6428,181*,54.01,7692,83.56,17,39,3,520,96
5,KC Sangakkara (SL),2010-2015,142,135,15,6356,169,52.96,7504,84.70,15,42,6,609,52
6,TM Dilshan (SL),2010-2016,159,155,14,6296,161*,44.65,7234,87.03,17,31,7,739,32
7,EJG Morgan (ENG),2010-2019,195,179,25,6241,148,40.52,6576,94.90,12,38,11,515,183
8,KS Williamson (NZ),2010-2019,149,142,14,6132,148,47.90,7494,81.82,13,39,5,559,49
9,MJ Guptill (NZ),2010-2019,157,155,16,5888,237*,42.35,6686,88.06,15,29,15,616,153


## Removing all-rounders and keeping what we need

For our training dataset, it is essential we give our model data of pure batsmen and bowlers only. So, according to Cricinfo's website we shall remove all players who are not pure batsmen.

In [110]:
#Removing all-rounders according to ESPN Cricinfo
df = df.drop([6, 11, 15, 22, 24, 32, 33])
df_batsmen = df[['Player', 'Runs', 'Ave', 'SR']].copy()
df_batsmen = df_batsmen.rename(columns = {'Runs':'Runs Scored', 'Ave':'Batting Average', 'SR':'Batting Strike Rate'})

### Now, we perform the same for the top 50 ODI bowlers of the last decade

This is what we get when we repeat the corresponding steps for them

In [63]:
page2 = requests.get('https://stats.espncricinfo.com/ci/engine/records/bowling/most_wickets_career.html?class=2;id=201;type=decade')

In [70]:
doc2 = lh.fromstring(page2.content)
tr_elements2 = doc2.xpath('//tr')

#create empty list
col2 = []
i = 0
for t in tr_elements2[0]:
    i+=1
    name = t.text_content()
    print ('%d:"%s"'%(i,name))
    col2.append((name,[]))
    
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements2[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=14:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col2[i][1].append(data)
        #Increment i for the next column
        i+=1

1:"Player"
2:"Span"
3:"Mat"
4:"Inns"
5:"Overs"
6:"Mdns"
7:"Runs"
8:"Wkts"
9:"BBI"
10:"Ave"
11:"Econ"
12:"SR"
13:"4"
14:"5"


In [74]:
Dict={title:column for (title,column) in col2}
df1=pd.DataFrame(Dict)
df1

,Player,Span,Mat,Inns,Overs,Mdns,Runs,Wkts,BBI,Ave,Econ,SR,4,5
0,SL Malinga (SL),2010-2019,162,158,1303.4,69,7129,248,6/38,28.74,5.46,31.5,6,8
1,Shakib Al Hasan (BDESH),2010-2019,131,129,1128.3,38,5337,177,5/29,30.15,4.72,38.2,8,2
2,Imran Tahir (SA),2011-2019,107,104,923.3,38,4297,173,7/45,24.83,4.65,32.0,7,3
3,MA Starc (AUS),2010-2019,85,85,719.1,39,3611,172,6/28,20.99,5.02,25.0,11,7
4,NLTC Perera (SL),2010-2019,159,151,942.5,27,5490,171,6/44,32.10,5.82,33.0,5,4
5,RA Jadeja (INDIA),2010-2019,145,142,1240.3,45,6046,171,5/36,35.35,4.87,43.5,6,1
6,TA Boult (NZ),2012-2019,89,89,814.0,58,4111,164,7/34,25.06,5.05,29.7,7,5
7,Saeed Ajmal (PAK),2010-2015,91,90,810.3,46,3439,157,5/24,21.90,4.24,30.9,5,2
8,TG Southee (NZ),2010-2019,116,114,971.1,63,5322,156,7/33,34.11,5.48,37.3,3,3
9,M Morkel (SA),2010-2018,94,92,759.2,38,3768,154,5/21,24.46,4.96,29.5,6,2


In [129]:
#Removing all-rounders according to ESPN Cricinfo
df1 = df1.drop([1, 5, 12, 19, 21, 35, 38, 42, 46])
df_bowlers = df1[['Player', 'Wkts', 'Ave', 'SR']].copy()
df_bowlers = df_bowlers.rename(columns = {'Wkts':'Wickets Taken', 'Ave':'Bowling Average', 'SR':'Bowling Strike Rate'})

### Now, before merging the two dataframes, we first add common columns

In [121]:
li = [0] * 43
#df_batsmen.insert(4, "Wickets Taken", li)
#df_batsmen.insert(5, "Bowling Average", li)
#df_batsmen.insert(6, "Bowling Strike Rate", li)
df_batsmen

,Player,Runs Scored,Batting Average,Batting Strike Rate,Wickets Taken,Bowling Average,Bowling Strike Rate
0,V Kohli (INDIA),11125,60.79,94.11,0,0,0
1,RG Sharma (INDIA),8249,53.56,90.63,0,0,0
2,HM Amla (SA),7265,49.76,89.11,0,0,0
3,AB de Villiers (SA),6485,64.20,109.76,0,0,0
4,LRPL Taylor (NZ),6428,54.01,83.56,0,0,0
5,KC Sangakkara (SL),6356,52.96,84.70,0,0,0
7,EJG Morgan (ENG),6241,40.52,94.90,0,0,0
8,KS Williamson (NZ),6132,47.90,81.82,0,0,0
9,MJ Guptill (NZ),5888,42.35,88.06,0,0,0
10,JE Root (ENG),5856,51.36,87.37,0,0,0


In [130]:
li = [0] * 40
df_bowlers.insert(1, "Runs Scored", li)
df_bowlers.insert(2, "Batting Average", li)
df_bowlers.insert(3, "Batting Strike Rate", li)
df_bowlers

,Player,Runs Scored,Batting Average,Batting Strike Rate,Wickets Taken,Bowling Average,Bowling Strike Rate
0,SL Malinga (SL),0,0,0,248,28.74,31.5
2,Imran Tahir (SA),0,0,0,173,24.83,32.0
3,MA Starc (AUS),0,0,0,172,20.99,25.0
4,NLTC Perera (SL),0,0,0,171,32.10,33.0
6,TA Boult (NZ),0,0,0,164,25.06,29.7
7,Saeed Ajmal (PAK),0,0,0,157,21.90,30.9
8,TG Southee (NZ),0,0,0,156,34.11,37.3
9,M Morkel (SA),0,0,0,154,24.46,29.5
10,R Ashwin (INDIA),0,0,0,150,32.91,40.1
11,DW Steyn (SA),0,0,0,145,24.80,31.4


In [131]:
#df_batsmen =  df_batsmen.set_index('Player')
#df_bowlers =  df_batdf_bowlerssmen.set_index('Player')

### Compiling and Merging The Two Dataframes

We will merge both dataframes to form our initial dataset. Then we can export it as a CSV file. Later, we will manually have to enter all the values that are remaining (currently set to zero). 

In [142]:
dataset = df_batsmen.append(df_bowlers)
l = [i for i in range(83)]
dataset.insert(0, "S. No", l)

In [144]:
dataset = dataset.set_index('S. No')

In [145]:
dataset

,Player,Runs Scored,Batting Average,Batting Strike Rate,Wickets Taken,Bowling Average,Bowling Strike Rate
S. No,,,,,,,
0,V Kohli (INDIA),11125,60.79,94.11,0,0,0
1,RG Sharma (INDIA),8249,53.56,90.63,0,0,0
2,HM Amla (SA),7265,49.76,89.11,0,0,0
3,AB de Villiers (SA),6485,64.20,109.76,0,0,0
4,LRPL Taylor (NZ),6428,54.01,83.56,0,0,0
5,KC Sangakkara (SL),6356,52.96,84.70,0,0,0
6,EJG Morgan (ENG),6241,40.52,94.90,0,0,0
7,KS Williamson (NZ),6132,47.90,81.82,0,0,0
8,MJ Guptill (NZ),5888,42.35,88.06,0,0,0


In [147]:
# Exporting as a CSV File - training_dataset.csv
dataset.to_csv(r'C:\Users\Anantk\Downloads\Projects\AI Project\training_dataset.csv', index = False)